In [ ]:
import os, sys

sys.path.append(os.path.dirname(os.path.dirname(os.path.abspath("__file__"))))
from nbafuns import *
from tqdm.notebook import trange, tqdm

from sklearn.preprocessing import MinMaxScaler
import torch
import torch.nn as nn
import torch.nn.functional as Func
from torch.utils.data import DataLoader, TensorDataset
# Only this extra line of code is required to use oneDNN Graph
torch.jit.enable_onednn_fusion(True)
torch.autograd.detect_anomaly = False
torch.autograd.profiler.emit_nvtx = False 
torch.autograd.profiler.profile = False
torch.autograd.gradcheck = False
torch.autograd.gradgradcheck = False

data_DIR = "../data/rapm/"
misc_DIR = "../data/misc/"
model_path = "../data/models/"
pbp_DIR = "../data/pbpdata/"
fig_DIR = "../figs/analysis/"

# %matplotlib widget

In [ ]:
if torch.cuda.is_available():
    print("CUDA is available!")
    print("Number of GPUs:", torch.cuda.device_count())
    print("Device name:", torch.cuda.get_device_name(0))
else:
    print("CUDA is not available.")

In [ ]:
device = torch.device("cpu")
device = torch.device("cuda:0")
device

## Data Pre Processing

In [ ]:
# loads possessions with odds
dfw = pd.read_parquet(data_DIR + "NBA_rapm_possessions_odds_2017_2024.parquet")
len(dfw)


In [ ]:
# random seed
rr = 11

In [ ]:
X = dfw[['margin', 'spread', 'secs']].values
y = dfw['win'].values

# sample the data
test_gid = dfw['gid'].sample(frac=0.8, random_state=rr).to_list()
dfw11 = dfw[dfw['gid'].isin(test_gid)]
# dfw12 = dfw11.query("secs <=120")
# for i in range(4):
#     dfw11  = pd.concat([dfw11,dfw12])
dfw1 = dfw11
dfw2 = dfw[dfw['gid'].isin(test_gid)]

# scale the data
scaler = MinMaxScaler()
smodel = scaler.fit(X)
Xs = smodel.transform(X)
X_train1 =  smodel.transform(dfw1[['margin', 'spread', 'secs']].values)
y_train1 = dfw1['win'].values
X_test1 =  smodel.transform(dfw2[['margin', 'spread', 'secs']].values)
y_test1 = dfw2['win'].values

## Cuda Tensors

In [ ]:
# convert to tensors
inputs = torch.tensor(Xs, dtype=torch.float32, device=device)
labels = torch.tensor(y, dtype=torch.float32, device=device).unsqueeze(1)
X_train = torch.tensor(X_train1, dtype=torch.float32, device=device)
y_train = torch.tensor(y_train1, dtype=torch.float32, device=device).unsqueeze(1)
X_test = torch.tensor(X_test1, dtype=torch.float32, device=device)
y_test = torch.tensor(y_test1, dtype=torch.float32, device=device).unsqueeze(1)

In [ ]:
h1 = 24
h2 = 24
torch.manual_seed(rr)
# Initialize the model
model = nn.Sequential(
    nn.Linear(3,h1),
    nn.ReLU(),
    nn.Linear(h1,h2),
    nn.ReLU(),
    nn.Linear(h2,1),
    nn.Sigmoid()
)
model = model.to(device)
criterion = nn.BCELoss()
# optimizer = torch.optim.RMSprop(model.parameters(),lr=1e-3)
optimizer = torch.optim.Adam(model.parameters(),lr=1e-3)

In [ ]:
t1 = perf_counter()
num_epochs = 500
losses = []
for epoch in trange(num_epochs,desc="epochs",leave=False):
    model.train()
    # optimizer.zero_grad()
    for param in model.parameters():
        param.grad = None
    outputs = model(inputs)
    loss = criterion(outputs, labels)
    loss.backward()
    optimizer.step()
    losses.append(loss.item())
t2 = perf_counter()

In [ ]:
print(round(t2-t1,1))

In [ ]:
with torch.no_grad():
    y_eval = model.forward(X_test)
    loss = criterion(y_eval,y_test)
print(loss)

In [ ]:
pred = model.forward(inputs).cpu()
x_out = dfw["secs"].values
y_out = pred.data.numpy()
fig,ax=plt.subplots(1,1,figsize=(5,4))
ax.plot(range(num_epochs),losses)
ax.set_xlabel("Epochs")
ax.set_ylabel("Loss")
ax.set_title(f"Layer 1: {h1:02d} Layer2: {h2:02d} Epochs: {num_epochs:03d}")
fig.tight_layout()

In [ ]:
fig,ax=plt.subplots(1,1,figsize=(5,4))
ax.plot(x_out,y_out,"x")
ax.set_xlim((3000,0))
ax.set_xlabel("Time Remaining in Game [s]")
ax.set_ylabel("Win Probability")
ax.set_title(f"Layer 1: {h1:02d} Layer2: {h2:02d} Epochs: {num_epochs:03d}")
fig.tight_layout()

In [ ]:
torch.save(model, f"./testing/model_epoch_{h1:02d}_{h2:02d}_{num_epochs:03d}.pt")

## Batch

In [ ]:
model = nn.Sequential(
    nn.Linear(3,h1),
    nn.ReLU(),
    nn.Linear(h1,h2),
    nn.ReLU(),
    nn.Linear(h2,1),
    nn.Sigmoid()
)
model = model.to(device)
criterion = nn.BCELoss()
# optimizer = torch.optim.RMSprop(model.parameters(),lr=1e-3)
optimizer = torch.optim.Adam(model.parameters(),lr=1e-3)
bs=4
dataset = TensorDataset(inputs, labels)
batch_size = int(len(dataset)/bs)+1
num_epochs = 50

In [ ]:
t3 = perf_counter()
dataloader = DataLoader(dataset, batch_size=batch_size)
t4 = perf_counter()
print(round(t4-t3,1))
losses = []
j = 0
for epoch in range(num_epochs):
    model.train()
    for i, batch in enumerate(dataloader):
        inputs_batch,labels_batch = batch
        for param in model.parameters():
            param.grad = None
        outputs = model(inputs_batch)
        loss = criterion(outputs, labels_batch)
        loss.backward()
        optimizer.step()
        print(i)
    clear_output(wait=True)
    print(f'Epoch {epoch}/{num_epochs}, Loss: {loss.item()}')
    losses.append(loss.item())
    j+=1
t5 = perf_counter()

In [ ]:
torch.save(model, f"./testing/model_batch_{h1:02d}_{h2:02d}_{num_epochs:03d}_batch{bs:02d}.pt")

In [ ]:
print(round(t4-t3,1))
print(round(t5-t4,1))

In [ ]:
with torch.no_grad():
    y_eval = model.forward(X_test)
    loss = criterion(y_eval,y_test)
print(loss)

In [ ]:
pred = model.forward(inputs).cpu()
x_out = dfw["secs"].values
y_out = pred.data.numpy()
fig,ax=plt.subplots(1,1,figsize=(5,4))
ax.plot(range(j),losses)
ax.set_xlabel("Epochs")
ax.set_ylabel("Loss")
ax.set_title(f"Layer 1: {h1:02d} Layer2: {h2:02d} Epochs: {num_epochs:03d}")
fig.tight_layout()

In [ ]:
fig,ax=plt.subplots(1,1,figsize=(5,4))
ax.plot(x_out,y_out,"x")
ax.set_xlim((3000,0))
ax.set_xlabel("Time Remaining in Game [s]")
ax.set_ylabel("Win Probability")
ax.set_title(f"Layer 1: {h1:02d} Layer2: {h2:02d} Epochs: {num_epochs:03d}")
fig.tight_layout()

In [ ]:
# torch.save(model, f"./testing/model_batch.pt")